In [3]:
from Cells import Cells
from MGFeatures import *
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import napari
from tifffile import imsave
import skimage.measure
import time
import seaborn as sns
import matplotlib.pyplot as plt
import cc3d

C:\Users\Gast-User\.conda\envs\MG\lib\site-packages\numba\core\decorators.py:253: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [4]:
def get_area(labels, labels_list): 
    areas = []
    for i in labels_list:
        area = np.count_nonzero(labels == i)
        areas.append(area)
    areas = np.array(areas)
    return areas

In [5]:
folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\cell_final'
heterochromatin_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\heterochromatin_final'
gold_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\gold_final'
lysosomes_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\lysosomes_final'
mitochondria_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\mitochondria_final'
nucleus_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\nucleus_final'
ER_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\er_final'
golgi_folder = r'Y:\MS293_Shun_Immunogold_030523\Segmentation\golgi_final'

In [9]:
start_time = time.time()
# Initialize an empty list to collect data for all 'sl_num'
all_mito_data = []

# Initialize an empty list to collect 'mito_area' values for each 'sl_num'
mito_area_values_list = []

for i in range(0, 5):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))
    mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)
    mito_coord = []
    region = regionprops(mito_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        mito_coord.append((min_row, min_col, max_row, max_col, sl_num))
    
    # Append mito_coord data for the current 'sl_num' to all_mito_data
    all_mito_data.extend(mito_coord)
    
    # Calculate 'mito_area' for the current 'sl_num' and collect the values
    mito_list = np.unique(mito_label)[1:]
    mito_area_values = get_area(mito_label, mito_list)
    mito_area_values_list.append(mito_area_values)

# Create the coord_mito DataFrame from the collected data
coord_mito = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])

# Combine 'mito_area' values into a single list
mito_area_values_combined = np.concatenate(mito_area_values_list)

# Assign 'mito_area' values to the coord_mito DataFrame
coord_mito['mito_area'] = mito_area_values_combined

# Initialize the 'mito_large' column with zeros
coord_mito['mito_large'] = 0

# Calculate 'mito_large' values
for y in range(len(coord_mito)):
    D = max(coord_mito['max_row'][y] - coord_mito['min_row'][y], coord_mito['max_col'][y] - coord_mito['min_col'][y])
    coord_mito.loc[y, 'mito_large'] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

In [10]:
coord_mito

,min_row,min_col,max_row,max_col,sl_num,mito_area,mito_large
0,3726,29344,3764,29376,0,944,38
1,3744,29272,3786,29324,0,1700,52
2,3860,28856,3912,28888,0,1296,52
3,3874,29368,3916,29410,0,1320,42
4,3880,29308,3906,29336,0,576,28
...,...,...,...,...,...,...,...
340,17931,12016,17963,12043,4,653,32
341,17962,7344,17992,7376,4,772,32
342,18035,12081,18079,12110,4,900,44
343,18122,12093,18178,12130,4,1540,56


In [11]:
coord_mito['volumen'] = coord_mito['mito_area'] * 200

In [12]:
coord_mito

,min_row,min_col,max_row,max_col,sl_num,mito_area,mito_large,volumen
0,3726,29344,3764,29376,0,944,38,188800
1,3744,29272,3786,29324,0,1700,52,340000
2,3860,28856,3912,28888,0,1296,52,259200
3,3874,29368,3916,29410,0,1320,42,264000
4,3880,29308,3906,29336,0,576,28,115200
...,...,...,...,...,...,...,...,...
340,17931,12016,17963,12043,4,653,32,130600
341,17962,7344,17992,7376,4,772,32,154400
342,18035,12081,18079,12110,4,900,44,180000
343,18122,12093,18178,12130,4,1540,56,308000


In [13]:
mito_labels = cc3d.connected_components(mito_stack, connectivity=26)

In [14]:
mito_labels.max()

324

In [15]:
def mito_to_3d(mito_labels, coord_mito):
    features_mito =  coord_mito.copy()
    features_mito['mito_3d'] = 0
    for j in range(5):
        for i in range(1, mito_labels[j].max()+1):
            layer = np.where(mito_labels[j] == i)
            if layer[0].size != 0: #coordinates of the first section of each of the cells
                for n in range(len(features_mito)):
                    if features_mito.loc[n, 'sl_num'] == j:
                        if (layer[0].min() >= features_mito['min_row'][n]) and (layer[0].max() <= features_mito['max_row'][n]) and (layer[1].min() >= features_mito['min_col'][n]) and (layer[1].max() <= features_mito['max_col'][n]):
                            features_mito['mito_3d'][n] = i
    return features_mito

In [17]:
start_time = time.time()
features_mito_5_3d = mito_to_3d(mito_labels, coord_mito)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)

C:\Users\Gast-User\AppData\Local\Temp\ipykernel_16972\803661094.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_mito['mito_3d'][n] = i


Elapsed time:  3093.943882703781


In [20]:
features_mito_5_3d.to_excel('features_mito_5_3d.xlsx')

AttributeError: 'NoneType' object has no attribute 'to_excel'

In [6]:
mitochondria_labels = mask_it(cell_labels, cell.img_from_tiles(mitochondria_folder))

In [7]:
mito_label = skimage.measure.label(mitochondria_labels, connectivity=2)

###  connectivity=26

In [22]:
mito_labels[0].max(), mito_labels[1].max(), mito_labels[2].max(), mito_labels[3].max(), mito_labels[4].max()

(49, 116, 182, 254, 324)

### connectivity=18

In [27]:
mito_labels = cc3d.connected_components(mito_stack, connectivity=18)

In [29]:
mito_labels[0].max(), mito_labels[1].max(), mito_labels[2].max(), mito_labels[3].max(), mito_labels[4].max()

(49, 116, 183, 257, 327)

### connectivity=6

In [33]:
mito_labels = cc3d.connected_components(mito_stack, connectivity=6)

In [34]:
mito_labels[0].max(), mito_labels[1].max(), mito_labels[2].max(), mito_labels[3].max(), mito_labels[4].max()

(49, 116, 187, 261, 333)

In [32]:
coord_mito = np.where(mito_labels[0] == 0)

In [31]:
coord_mito

(array([18115, 18115, 18115, 18115, 18115, 18115, 18115, 18116, 18116,
        18116, 18116, 18116, 18116, 18116, 18116, 18116, 18117, 18117,
        18117, 18117, 18117, 18117, 18117, 18117, 18117, 18117, 18117,
        18118, 18118, 18118, 18118, 18118, 18118, 18118, 18118, 18118,
        18118, 18118, 18118, 18118, 18119, 18119, 18119, 18119, 18119,
        18119, 18119, 18119, 18119, 18119, 18119, 18119, 18119, 18119,
        18119, 18120, 18120, 18120, 18120, 18120, 18120, 18120, 18120,
        18120, 18120, 18120, 18120, 18120, 18120, 18120, 18120, 18121,
        18121, 18121, 18121, 18121, 18121, 18121, 18121, 18121, 18121,
        18121, 18121, 18121, 18121, 18121, 18121, 18122, 18122, 18122,
        18122, 18122, 18122, 18122, 18122, 18122, 18122, 18122, 18122,
        18122, 18122, 18122, 18122, 18123, 18123, 18123, 18123, 18123,
        18123, 18123, 18123, 18123, 18123, 18123, 18123, 18123, 18123,
        18123, 18123, 18123, 18124, 18124, 18124, 18124, 18124, 18124,
      

# Nuclear calculations

In [ ]:
start_time = time.time()
nuc_stack = np.zeros(shape = (5, 37440, 35168))
all_nuc_data = []
for i in range(0,5):
    cell = Cells(i)
    imageshape = cell.get_imageshape(folder) 
    cell_image = cell.img_from_tiles(folder)
    cell_labels = cell.label_cells(cell_image)
    nucleus_labels = mask_it(cell_labels, cell.img_from_tiles(nucleus_folder))
    nuc_label = skimage.measure.label(nucleus_labels, connectivity=2)
    nuc_coord = []
    region = regionprops(nuc_label)
    for r in region:
        sl_num = i
        min_row, min_col, max_row, max_col = r.bbox
        nuc_coord.append((min_row, min_col, max_row, max_col, sl_num))
    all_nuc_data.extend(mito_coord)
    nuc_stack[i] = nuc_label
coord_nuc = pd.DataFrame(all_mito_data, columns=["min_row", "min_col", "max_row", "max_col", 'sl_num'])
#       coord_mito['sl_num'][r] = i
unique_sl_nums = coord_mito['sl_num'].unique()
for sl_num in unique_sl_nums:
    mito_list = np.unique(coord_mito[coord_mito['sl_num'] == sl_num]['sl_num'])
    mito_area_values = get_area(mito_label, mito_list)
    coord_mito.loc[coord_mito['sl_num'] == sl_num, 'mito_area'] = mito_area_values
coord_mito['mito_large'] = 0
for y in range(len(coord_mito)):
    D = max(coord_mito['max_row'][y] - coord_mito['min_row'][y], coord_mito['max_col'][y] - coord_mito['min_col'][y])
    coord_mito['mito_large'][y] = D
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time)